# Adaptive Agent Deployment with Azure AI Foundry

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder
import os

In [ ]:
# Setup project client
project = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

In [ ]:
# Define Adaptive Card template
adaptive_card = {
    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
    "type": "AdaptiveCard",
    "version": "1.5",
    "body": [
        {"type": "TextBlock", "text": "Welcome!", "size": "Large"},
        {"type": "Input.Text", "id": "userInput", "placeholder": "Ask me something..."}
    ],
    "actions": [{"type": "Action.Submit", "title": "Send"}]
}

In [ ]:
# Create agent
agent = project.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="adaptive-agent",
    instructions="You are a helpful assistant using Adaptive Cards to guide users.",
    tools=[{"type": "adaptiveCard", "card": adaptive_card}]
)

In [ ]:
# Create thread and send message
thread = project.agents.threads.create()
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How can I reset my password?"
)

In [ ]:
# Run agent and get response
run = project.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.run_id == run.id and msg.text_messages:
        print(f"{msg.role}: {msg.text_messages[-1].text.value}")